<a href="https://colab.research.google.com/github/dfragos/EMOS-2024/blob/main/EMOS_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Βήμα 1: Φόρτωση Πακέτων και Δεδομένων από την Eurostat 

In [ ]:
# Εγκατάσταση και φόρτωση πακέτων
if (!require("eurostat")) install.packages("eurostat")
if (!require("dplyr")) install.packages("dplyr")
if (!require("tidyr")) install.packages("tidyr")
if (!require("stringr")) install.packages("stringr")

library(eurostat)
library(dplyr)
library(tidyr)
library(stringr)

# Λήψη δεδομένων από την Eurostat για την ανεργία
unemployment_data <- get_eurostat("une_rt_m", time_format = "date")

# Επισκόπηση των δεδομένων
head(unemployment_data)

Βήμα 2: Εντοπισμός και Διαχείριση Ελλειπουσών Τιμών 

In [ ]:
# Επισκόπηση ελλειπουσών τιμών
summary(unemployment_data)

# Αντικατάσταση NA με 0 (εφόσον αυτό είναι λογικό) ή άλλες στρατηγικές
unemployment_data <- unemployment_data %>%
  mutate(values = ifelse(is.na(values), 0, values))

Βήμα 3: Εντοπισμός και Διαχείριση Ακραίων Τιμών 

In [ ]:
# Ανάλυση boxplot για ανίχνευση ακραίων τιμών
boxplot(unemployment_data$values)

# Αντικατάσταση ακραίων τιμών με IQR-based μέθοδο
q1 <- quantile(unemployment_data$values, 0.25)
q3 <- quantile(unemployment_data$values, 0.75)
iqr <- q3 - q1
lower_bound <- q1 - 1.5 * iqr
upper_bound <- q3 + 1.5 * iqr

# Ενημέρωση ακραίων τιμών
unemployment_data <- unemployment_data %>%
  mutate(values = ifelse(values < lower_bound | values > upper_bound, NA, values))

Βήμα 4: Καθαρισμός Κατηγορικών Τιμών 

In [ ]:
# Καθαρισμός λανθασμένων κατηγορικών τιμών
unemployment_data <- unemployment_data %>%
  filter(!is.na(geo)) %>%  # Αφαίρεση τιμών χωρίς γεωγραφική κατηγορία
  mutate(geo = str_to_upper(geo))  # Εναρμόνιση με κεφαλαία γράμματα

Βήμα 5: Εντοπισμός και Αφαίρεση Διπλών Εγγραφών 

In [ ]:
# Αφαίρεση διπλών εγγραφών
unemployment_data <- unemployment_data %>%
  distinct()

Βήμα 6: Συμπλήρωση Δεδομένων

In [ ]:
# Συμπλήρωση τιμών που λείπουν
unemployment_data <- unemployment_data %>%
  group_by(geo, sex, age) %>%
  mutate(values = ifelse(is.na(values), mean(values, na.rm = TRUE), values))

Βήμα 7: Καθαρισμός Κειμένου 

In [ ]:
# Καθαρισμός τυχόν ανεπιθύμητων χαρακτήρων
unemployment_data <- unemployment_data %>%
  mutate(geo = str_squish(geo),
         sex = str_squish(sex),
         age = str_squish(age))

Βήμα 8: Αντιμετώπιση Ασυνεπειών 

In [ ]:
# Έλεγχος για ασυνεπείς ή μη λογικές τιμές
unemployment_data <- unemployment_data %>%
  filter(values >= 0)  # Απομάκρυνση αρνητικών τιμών

Βήμα 9: Έλεγχος Μορφοποίησης 

In [ ]:
# Μετατροπή όλων των πεδίων σε κατάλληλη μορφή
unemployment_data <- unemployment_data %>%
  mutate(values = as.numeric(values),
         time = as.Date(time))

Βήμα 10: Χειρισμός Αρνητικών, Μη Λογικών ή Αντιφατικών Τιμών 

In [ ]:
# Αφαίρεση μη λογικών τιμών - Ποσοστά ανεργίας πρέπει να είναι μεταξύ 0-100

unemployment_data <- unemployment_data %>%
  filter(values >= 0 & values <= 100)  

Βήμα 11: Αποθήκευση των δεδομένων 

In [ ]:
# Αποθήκευση σε αρχείο CSV
write.csv(unemployment_data, "cleaned_unemployment_data.csv", row.names = FALSE)


******* bonus
# Εγκατάσταση arrow (αν δεν είναι ήδη εγκατεστημένο)
if (!require("arrow")) install.packages("arrow")

# Φόρτωση βιβλιοθήκης
library(arrow)

# Αποθήκευση των καθαρισμένων δεδομένων σε αρχείο Parquet
write_parquet(unemployment_data, "cleaned_unemployment_data.parquet")